# ch 3. tf-idf

이번 챕터에서는 대표적인 빈도 수 기반의 NLP 기법 중 하나인 tf-idf에 대해서 배워보겠습니다. tf-idf란 term-frequency & inverse document frequency의 약자로, 특정 문서를 벡터화 할 때 사용되는 단순하면서도 효과적인 기법입니다. tf-idf를 이용하면 문서 내에 등장하는 단어들에 중요도를 매길 수 있습니다. 이를 통해 유사한 문서 찾기, 문서 내 중요한 단어 파악하기 등의 기능을 구현할 수 있습니다. 중요도를 매긴 다는 것이 어떤 의미인지 예시를 통해서 알아보겠습니다.

### Term Frequency(TF)
먼저 이대호 선수에 관련된 뉴스 기사 한편을 토큰화 한 뒤, 문서 내에서 토큰의 등장 빈도를 세어 보겠습니다. 문서 안에서 여러번 등장한 "이대호", "롯데"와 같은 토큰은 한번 등장한 "손아섭", "NC" 같은 토큰 보다는 더 중요한 토큰이겠죠? 이처럼 문서 내에서 자주 등장한 토큰에 가중치를 부여하기 위한 지표가 tf(term frequency)입니다. 

$$tf(t, d)=문서\;내\;특정\;토큰\;등장\;빈도$$

$$tf(\text{"시즌"}, d_{0})=6$$

$$tf(\text{"시즌"}, d_{1})=5$$

이 외에도 tf를 계산하는 다양한 방식들이 있습니다. 특정 문서 내에서 등장하면 1, 등장하지 않았으면 0으로 할당하는 방법, 등장 빈도 수에 로그를 취하는 방법, 가장 많이 등장한 단어에 대한 등장 비율로 계산할 수도 있어서, 구현체에 따라 다릅니다.

- boolean frequency: tf(t, d)=t가 d에 한번이라도 등장하면 1, 아니면 0
- log scaled frequency: tf(t, d)=log(f(t, d) + 1)
- augmented frequency: 최빈 단어를 분모로 target 단어의 TF를 나눈 값으로, 일반적으로는 문서의 길이가 상대적으로 길 경우, 단어 빈도값을 조절하기 위해 사용

$$tf(t,d)=0.5+\frac{0.5\times f(t,d)}{max{f(w,d): w\in d}}$$

$$tf(\text{"시즌"},\;d_{1})=0.5+\frac{0.5*5}{16}\approx0.6562$$

정리해보면 tf는 하나의 문서 내에서 특정 단어가 얼마나 자주 등장했는지를 측정하는 지표이며, 세부 구현은 여러가지가 있다 정도로 기억하고 넘어가겠습니다.

### Document Frequency(DF)
두 기사의 TF를 보면 공통적으로 "이대호", "롯데"와 같은 토큰이 자주 등장하는 것을 알 수 있습니다. 그런데 "꿈", "불혹" 같은 토큰은 첫번째 기사에, "소프트뱅크" 토큰은 두번째 기사에만 등장했습니다. 이들 토큰은 비록 문서 내 등장 빈도는 적지만, 다른 문서에는 잘 등장하지 않는 문서의 특징을 잘 대변하는 토큰이라고 할 수 있습니다. 

특정 단어가 얼마나 전체 문서에서 흔하게 등장하는 지를 측정하는 지표가 df(document frequency)입니다. 이는 전체 문서에서 특정 단어가 등장한 문서 수를 집계한 수 입니다. 위 예시에서는 "이대호" 는 모든 문서에 등장하므로 df가 2, "불혹"이라는 단어는 1번 문서에만 등장하므로 df는 1이 됩니다.

$$df(t):특정\;토큰이\;등장한\;문서의\;수$$

$$df(\text{"시즌"})=2$$

$$df(\text{"불혹"})=1$$

### Inverse Document Frequency(IDF)
그런데 우리가 원하는 건 모든 문서에 등장하는 흔한 단어가 아니라, 특정 문서에만 등장하는 희귀한 단어입니다. df 값에 역수를 취해주면 특정 토큰이 얼마나 희귀한 단어인지를 나타내는 idf 값이 됩니다. 수식은 아래와 같습니다. 

$$idf(t, D)=log(\frac{|D|+1}{df(t)+1})$$



먼저 전체 문서의 집합을 D로 표기합니다. |D|는 전체 문서의 개수입니다. 분자는 특정 토큰 t를 포함한 문서 집합의 개수를 의미합니다. 즉, 전체 문서 개수를 특정 토큰을 포함한 문서 개수로 나눠준 값입니다. 

분모와 분자에 1씩 더해주는 것은 학습 데이터 셋에 등장한 적 없는 새로운 단어가 등장했을 때, df 값이 0이 되어 division by zero가 일어나는 현상을 방지하기 위함입니다. 이 때, 마치 모든 단어를 포함한 문서가 하나 있다고 가정하고 분모와 분자에 1씩 더해줍니다.

마지막으로 로그를 씌워주겠습니다. 현재 우리가 수집한 2022년 한국 야구 뉴스 기사는 총 10만건 정도 됩니다. 만약 특정 기사에 딱 한번만 등장한 단어가 있다면 idf 값은 10만이 되버리겠죠? 이렇게 지나치게 큰 값이 나오는 상황을 방지하기 위해서 log를 한번 씌워줍니다.

예시 데이터 셋으로 한번 계산해보겠습니다.

$$idf(\text{"시즌"}, D)=log(\frac{2+1}{2+1})=0$$

### TF-IDF
tf와 idf 값을 조합하여 tf-idf 값을 계산합니다. tf-idf 값이 높다는 의미는 특정 문서 내에서 자주 등장하면서도, 다른 문서에는 잘 등장하지 않는 중요한 토큰임을 의미합니다.

$$\text{TF-IDF}(t, d, D)=tf(t,d)*idf(t,D)$$

## TF-IDF를 이용한 문서 벡터화
각 단어별 tf-idf 값을 계산할 수 있으면, 이제 문서를 벡터로 표현할 수 있습니다. 한번 간단한 예제를 풀어보겠습니다.

### 문서 단어 행렬(DTM)
아래와 같이 네 개의 문서가 있습니다. 이를 토큰화 한 뒤, 행렬로 표기하면 아래와 같습니다. 표 안의 숫자가 곧 tf가 됩니다.

- 문서1 : 먹고 싶은 사과
- 문서2 : 먹고 싶은 바나나
- 문서3 : 길고 노란 바나나 바나나
- 문서4 : 저는 과일이 좋아요

||과일이|길고|노란|먹고|바나나|사과|싶은|저는|좋아요|
|-|-|-|-|-|-|-|-|-|-|
|문서1|0|0|0|1|0|1|1|0|0|
|문서2|0|0|0|1|1|0|1|0|0|
|문서3|0|1|1|0|2|0|0|0|0|
|문서4|1|0|0|0|0|0|0|1|1|

### tf-idf 계산

각 단어별 idf를 계산하면 아래와 같습니다.

|단어|IDF(역 문서 빈도)|
|-|-|
|과일이|ln(4+1/(1+1)) = 0.9162|
|길고|ln(4+1/(1+1)) = 0.9162|
|노란|ln(4+1/(1+1)) = 0.9162|
|먹고|ln(4+1/(2+1)) = 0.5108|
|바나나|ln(4+1/(2+1)) = 0.5108|
|사과|ln(4+1/(1+1)) = 0.9162|
|싶은|ln(4+1/(2+1)) = 0.5108|
|저는|ln(4+1/(1+1)) = 0.9162|
|좋아요|ln(4+1/(1+1)) = 0.9162|

tf-idf를 이용하여 각 문서를 벡터로 나타내면 아래와 같습니다.

||과일이|길고|노란|먹고|바나나|사과|싶은|저는|좋아요|
|-|-|-|-|-|-|-|-|-|-|
|문서1|0|0|0|0.5108|0|0.9162|0.5108|0|0|
|문서2|0|0|0|0.5108|0.5108|0|0.5108|0|0|
|문서3|0|0.9162|0.9162|0|1.8324|0|0|0|0|
|문서4|0.9162|0|0|0|0|0|0|0.9162|0.9162|

## 정리 
이번 챕터에서는 기본적인 NLP 알고리즘 중 하나인 tf-idf 개념에 대해서 학습해보았습니다. 그리고 tf-idf를 이용해서 문서를 벡터로 나타낼 수 있었습니다. 

tf-idf로 벡터화 시킨 문서는 유사도를 측정하여 유사한 문장을 찾는다던가, 분류 모델을 학습시켜서 문서의 종류를 분류하는데 사용됩니다. 최근에는 딥러닝을 이용하여 문서를 벡터화 시키는 것이 대세지만, tf-idf는 충분히 그 간결함으로 많이 사용되는 기본 NLP 알고리즘입니다.

이어지는 챕터에서는 scikit-learn의 Tfidf 구현체를 이용하여 실제 2022년 야구 뉴스 데이터 셋을 벡터로 표현하고, tf-idf를 활용한 문서 검색을 구현해보겠습니다.